In [ ]:
import os
import numpy as np
import tensorflow as tf
print(tf.__version__)
from scipy.io import loadmat
import h5py
import functools

# %matplotlib widget
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update({'font.size': 16})
mpl.rcParams.update({'font.family': 'serif'})

from utils_data import get_ae_data_acfiles_v1

TEST_NAME = 'p4581'
DATA_DIR = '/misc/projects/criticalstress-ldrd/cwj/public/Lab_AE/data_lab_p4581/'

In [ ]:
# AE data info
Syncfile = loadmat(DATA_DIR+'ACTimeVector_4581.mat')
acsettings = loadmat(DATA_DIR+'p4581_AE.mat')
ACfile_DIR = DATA_DIR+'ACFiles/'

# number of superframes per file
numSFpfile = int(acsettings['numFrames'][0]/2)
# number of WF per superframe and per channel
numWFpSFpCH = int(acsettings['numAcqs'][0])
# number of WF per file and per channel
numWFpfilepCH = numSFpfile * numWFpSFpCH
# number of channels
numCHR = len(acsettings['channels2save'][0])
# waveform length
WFlength = int(acsettings['Nsamples'][0])
# true number of points per file
TruenumPointspfilepCH = numWFpfilepCH * WFlength

print('numSFpfile', numSFpfile)
print('numWFpSFpCH', numWFpSFpCH)
print('numWFpfilepCH', numWFpfilepCH)
print('numCHR', numCHR)
print('WFlength', WFlength)
print('TruenumPointspfilepCH', TruenumPointspfilepCH)

In [ ]:
seg_split = [
    [2025.0, 2275.0, '2mpa_stepup', 220, 410], 
    [2325.0, 2525.0, '3mpa_stepup', 445, 600], 
    [2575.0, 2875.0, '4mpa_stepup', 630, 860], 
    [2925.0, 3175.0, '5mpa_stepup', 895, 1085], 
    [3225.0, 3475.0, '6mpa_stepup', 1120, 1310], 
    [3525.0, 3775.0, '7mpa_stepup', 1345, 1535], 
    [3825.0, 4075.0, '8mpa_stepup', 1565, 1760], 
    [4125.0, 4275.0, '7mpa_stepdown', 1790, 1910], 
    [4325.0, 4475.0, '6mpa_stepdown', 1940, 2060], 
    [4525.0, 4675.0, '5mpa_stepdown', 2090, 2210], 
    [4725.0, 4825.0, '4mpa_stepdown', 2240, 2320], 
]

In [ ]:
for i_seg_split in seg_split:
    start_time = i_seg_split[0]
    end_time = i_seg_split[1]
    seg_name = i_seg_split[2]
    start_file_number = i_seg_split[3]
    end_file_number = i_seg_split[4]
    print('\n', seg_name, start_time, end_time)
    
    with h5py.File(DATA_DIR+TEST_NAME+'_ae_data_'+seg_name+'.hdf5', "w") as f:
      dset_ae = f.create_dataset('ae', (0, numCHR), maxshape=(None, numCHR), dtype='<i2')
      dset_time = f.create_dataset('time', (0,), maxshape=(None,), dtype='<f8')
      dset_time_seg = f.create_dataset('time_seg', (0,), maxshape=(None,), dtype='<f8')

      dset_time_seg.attrs['WFlength'] = WFlength
      dset_time_seg.attrs['numWFpfilepCH'] = numWFpfilepCH
      dset_time_seg.attrs['TruenumPointspfilepCH'] = TruenumPointspfilepCH
      
      for filenumber in range(start_file_number, end_file_number):
        dset_ae.resize(dset_ae.shape[0]+TruenumPointspfilepCH, axis=0)
        dset_time.resize(dset_time.shape[0]+TruenumPointspfilepCH, axis=0)
        dset_time_seg.resize(dset_time_seg.shape[0]+numWFpfilepCH, axis=0)

        wf_ae, time_ae, time_ae_seg = get_ae_data_acfiles_v1(ACfile_DIR, acsettings, Syncfile, filenumber)

        print('  writing ae file', filenumber, time_ae[0], time_ae[-1], wf_ae.shape, time_ae.shape, time_ae_seg.shape)

        dset_ae[-TruenumPointspfilepCH:,:] = wf_ae
        dset_time[-TruenumPointspfilepCH:] = time_ae
        dset_time_seg[-numWFpfilepCH:] = time_ae_seg